# Big Data y Machine Learning (UBA) 2025
## Clase 3 - Parte 2 - Kernels

**Objetivo:**
Que se familiaricen con el segundo método no paramétrico - Kernels - para la estimación de la distribución de densidad de una variable aleatoria.

### Métodos no paramétricos
El objetivo es predecir distribución de una variable de interés 
- 𝑌 variable aleatoria de interés
- 𝑓(𝑌) distribución de densidad 𝑌

##### Métodos
- Breve repaso de Histogramas
- Kernels con Sckit-learn
    -  Tipo de funciones de kernels
    -  Opciones de Kernels: Ancho de banda $h$
    -  Simulación de datos: Sesgo de la estimación no parametrica de Kernels
- Kernels con Seaborn


In [ ]:
# Importamos paquetes
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from scipy.stats import norm #para crear datos de distribucion normal con otro modulo

import seaborn as sns
from sklearn.neighbors import KernelDensity

### Breve repaso de Histogramas


Estimamos la distribucion de densidad $f(Y)$ una variable aleatoria Y, con la siguiente aproximación no parametrica:

$$
\hat{f}(y) = \frac{M}{n} ∑^𝑛_i I(𝑌_𝑖 \in B_l)  
$$
Con $B_l$ barra (bin) $l$-ésimo

Podemos usar el atributo `hist` de Matplotlib. Ver documentación [acá](https://matplotlib.org/stable/api/_as_gen/matplotlib.pyplot.hist.html)

In [ ]:
# Generamos datos
np.random.seed(20)
X = np.concatenate([np.random.normal(0,1,500), np.random.normal(5,1,500)]).reshape(-1,1)
X

In [ ]:
X_df=pd.DataFrame(X)

In [ ]:
X_df.describe().round(2)

In [ ]:
# Grafico
plt.figure(figsize=(10,6))
plt.hist(X, bins=30, alpha=0.5, color='blue', label='Histograma')
plt.xlabel('Valores')
plt.ylabel('Frecuencia')

# Agregamos línea vertical con la media
mean_value = np.mean(X)
plt.axvline(mean_value, color='red', linestyle='dashed', linewidth=1, label='Media')
plt.legend()  # Show legend with label for the mean line
plt.show()

## Kernels

Kernel:
A cada observación le estima una pequeña función de densidad y suma todas las pequeñas funciones

$$
𝑓(𝑦_0)= \frac{1}{n} ∑^𝑛_i  \frac{1}{h} 𝐾(\frac{𝑌_𝑖−𝑦_0}{h}) 
$$

- $K(z)$  función Kernel continua (y generalmente) simétrica 
- $h$ ancho de banda (smoothing bandwidth) --> Controla qué tan “suave” es la densidad 


Vamos a usar el [módulo neighbors de Scikit learn](https://scikit-learn.org/stable/modules/generated/sklearn.neighbors.KernelDensity.html)

Para estimar una densidad usando kernels tenemos la siguiente función: 

<code> sklearn.neighbors.KernelDensity(*, bandwidth=1.0, algorithm='auto', kernel='gaussian', metric='euclidean', atol=0, rtol=0, breadth_first=True, leaf_size=40, metric_params=None)</code>

donde algunos parámetros importantes son:
- <code>bandwidth</code> (valor por default: 1.0)
- <code>kernel</code> (valor por default: 'gaussian')

Scikit learn nos permite cambiar el kernel y probar varios y cuál ajusta mejor a los datos

In [ ]:
# Grafico
plt.figure(figsize=(10,6))
plt.hist(X, bins=30, density=True, alpha=0.3, color='blue', label='Histograma') # mantenemos el histogramas para comparar

# Rango de valores para eje x (para graficar la funcion de Kernel)
X_plot = np.linspace(min(X), max(X), 1000).reshape(-1,1)

# Estimamos la funcion de Kernel Gaussiana y todas sus opciones de default
kde = KernelDensity().fit(X)
    
# Usar la KDE para estimar la densidad para cada valor de X
log_densities = kde.score_samples(X_plot)
densities = np.exp(log_densities)
    
# Grafico de kernel
plt.plot(X_plot[:,0], densities, color='red', label=f'Gaussian Kernel')

# Agregamos línea vertical con la media
mean_value = np.mean(X)
plt.axvline(mean_value, color='green', linestyle='dashed', linewidth=1, label='Media')

plt.legend()
plt.title('Estimación con Kernel Gaussiano')

#### Tipos de kernels (disponibles en Scikit learn)

In [ ]:
# Kernels
kernels = ["gaussian", "tophat", "epanechnikov", "exponential", "linear", "cosine"] 
  
# Figura con 3 filas y 2 columnas
fig, ax = plt.subplots(3, 2) 
# Tamaño de la figura
fig.set_figheight(15) 
fig.set_figwidth(10)   
# Título 
fig.suptitle("Tipos de kernels") 

# 1D array de valores de x para graficar la distribución 
x_plot = np.linspace(-6, 6, 1000) # 1000 valores de -6 a 6 separados con la misma distancia entre sí
x_plot = x_plot.reshape(-1,1) # formato 2D array (necesario para scikit learn)
x_orig = np.zeros((1, 1)) # punto (0,0)
  
# Graficamos usando los distintos kernels 
for i, kernel in enumerate(kernels): 
    # Ajustamos el modelo 
    kde = KernelDensity(kernel=kernel).fit(x_orig) # usamos el punto (0,0)
    # log de la densidad de probabilidad (PDF)
    log_dens = kde.score_samples(x_plot) 
      
    # Distribuciones 
    ax[i // 2, i % 2].fill(x_plot[:, 0], np.exp(log_dens)) 
    # i//2 nos permite referirnos a la fila del subplot, e i%2 nos permite referirnos a la columna
    # Título y labels de los subplots 
    ax[i // 2, i % 2].set_title(kernel.capitalize()) 
    ax[i // 2, i % 2].set_xlim(-3, 3) 
    ax[i // 2, i % 2].set_ylim(0, 1) 
    ax[i // 2, i % 2].set_ylabel("Densidad") 
    ax[i // 2, i % 2].set_xlabel("x") 
plt.show()

De la misma forma, en un gráfico

In [ ]:
# Kernels
kernels = ["gaussian", "tophat", "epanechnikov", "exponential", "linear", "cosine"] 
  
# Grafico
plt.figure(figsize=(10,6))

for k in kernels:
    # Ajustamos el modelo 
    kde = KernelDensity(kernel=k).fit(x_orig) # usamos el punto (0,0)
    # log de la densidad de probabilidad (PDF)
    log_dens = kde.score_samples(x_plot) 
    
    # Graficar la estimacion para cada kernel
    plt.plot(x_plot[:,0], np.exp(log_dens), label=f'{k.capitalize()} Kernel')

plt.legend()
plt.title('Estimación con diferentes Kernels')
plt.show()

Continuamos con el ejemplo de la variable X creada (en la clase pasada) probando los distintos tipos de kernels

In [ ]:
# Lista de kernels a probar
kernels = ["gaussian", "tophat", "epanechnikov", "exponential", "linear", "cosine"] 

# Grafico
plt.figure(figsize=(10,6))
plt.hist(X, bins=30, density=True, alpha=0.3, color='blue', label='Histograma')

for k in kernels:
    kde = KernelDensity(kernel=k).fit(X)
    
    # Usar la KDE para estimar la densidad para cada valor de X
    log_densities = kde.score_samples(X_plot)
    densities = np.exp(log_densities)
    
    # Graficar para cada kernel
    plt.plot(X_plot[:,0], densities, label=f'{k.capitalize()} Kernel')

plt.legend()
plt.title('Estimación con diferentes Kernels')

#### Opciones de Kernels: Ancho de banda $h$
Ahora veamos qué ocurre si para un mismo kernel, cambiamos los **anchos de banda**

In [ ]:
# Anchos de banda
bandwidths = [0.5, 0.75, 1, 1.25, 1.5, 1.75] 
  
# Figura con 3 filas y 2 columnas
fig, ax = plt.subplots(3, 2) 
# Tamaño de la figura
fig.set_figheight(15) 
fig.set_figwidth(10)   
# Título 
fig.suptitle('Kernel Gaussiano, con distintos anchos de banda')

# Graficamos usando los distintos kernels 
for i, bw in enumerate(bandwidths): 
    # Ajustamos el modelo 
    kde = KernelDensity(kernel='gaussian', bandwidth=bw).fit(x_orig) # usamos el punto (0,0)
    # log de la densidad de probabilidad (PDF)
    log_dens = kde.score_samples(x_plot) 
      
    # Distribuciones 
    ax[i // 2, i % 2].fill(x_plot[:, 0], np.exp(log_dens)) 
    # i//2 nos permite referirnos a la fila del subplot, e i%2 nos permite referirnos a la columna
    # Título y labels de los subplots 
    ax[i // 2, i % 2].set_title('Kernel Gaussiano con bandwidth='+str(bw)) 
    ax[i // 2, i % 2].set_xlim(-3, 3) 
    ax[i // 2, i % 2].set_ylim(0, 1) 
    ax[i // 2, i % 2].set_ylabel('Densidad') 
    ax[i // 2, i % 2].set_xlabel('x') 
plt.show()

In [ ]:
# Anchos de banda
bandwidths = [0.5, 0.75, 1, 1.25, 1.5, 1.75] 
  
# Grafico
plt.figure(figsize=(10,6))

for bw in bandwidths:
    # Ajustamos el modelo 
    kde = KernelDensity(kernel='gaussian', bandwidth=bw).fit(x_orig) # usamos el punto (0,0)
    # log de la densidad de probabilidad (PDF)
    log_dens = kde.score_samples(x_plot) 
    
    # Graficar la estimacion para cada kernel
    plt.plot(x_plot[:,0], np.exp(log_dens), label='Kernel Gaussiano con bandwidth='+str(bw))

plt.legend()
plt.title('Kernel Gaussiano, con distintos anchos de banda') 
plt.show()

In [ ]:
# Anchos de banda
bandwidths = [0.5, 0.75, 1, 1.25, 1.5, 1.75] 

# Grafico
plt.figure(figsize=(10,6))
plt.hist(X, bins=30, density=True, alpha=0.5, color='blue', label='Histograma')

for bw in bandwidths:
    kde = KernelDensity(kernel='gaussian', bandwidth=bw).fit(X)
    
    # Usar la KDE para estimar la densidad para cada valor de X
    log_densities = kde.score_samples(X_plot)
    densities = np.exp(log_densities)
    
    # Graficar para cada kernel
    plt.plot(X_plot[:,0], densities, label='Bandwidth='+str(bw))

plt.legend()
plt.title('Estimación de Kernel Gaussiano con diferentes ancho de banda')

### Simulación de datos: Sesgo de la estimación no parametrica de Kernels
Ahora veamos un ejemplo donde creamos datos ficticios, esto implica que conocemos la verdadera forma en la que se generan los datos, para comparar la estimación no paramétrica de Kernels y su aproximación a la verdadera función de densidad. Se puede demostrar formalmente, que la estimación no paramétrica de Kernels (e histograma) es *sesgada*. Por lo que, aquí estamos ilustrando ese concepto.

In [ ]:
# Creamos una distribución
n = 10000000
np.random.seed(100)
X = np.concatenate((np.random.normal(0, 1, int(0.6 * n)), np.random.normal(10, 1, int(0.4 * n)))) 
# Creamos X concatenando datos de dos distribuciones normales
# primero 60 datos de una distribución normal con media 0 y desvío 1
# luego, 40 datos de una normal con media 10 y desvío 1
X = X.reshape(-1,1)

X_plot = np.linspace(-5, 15, 1000).reshape(-1,1)
# Usaremos X para estimar la densidad y calcularemos la densidad para los puntos de X_plot 

# Calcular la "verdera" densidad para los puntos X_plot
true_density = 0.6 * norm(0, 1).pdf(X_plot[:, 0]) + 0.4 * norm(10, 1).pdf(X_plot[:, 0]) 
  
# Gráfico
fig, ax = plt.subplots() 
  
# Gráfico de la verdadera densidad 
ax.fill( 
    X_plot[:, 0], true_density,  
    fc='black', alpha=0.2,  
    label='Verdadera Distribución'
) 
  
# Estimar la densidad de X usando kernel gaussiano y bandwidth de 0.5 
kde = KernelDensity(kernel='gaussian', bandwidth=0.5).fit(X) 
# Log de la PDF 
log_dens = kde.score_samples(X_plot) 
  
# Densidad 
ax.plot( 
    X_plot[:, 0], np.exp(log_dens), 
    color='blue', 
    linestyle='-', 
    label='Densidad con kernel Gaussiano'
)  
ax.set_xlim(-4, 15) 
ax.set_ylim(0, 0.3) 
#ax.grid(True) 
ax.legend(loc='upper right')
plt.title('Sesgo de la estimación por Kernel') 

plt.show()

Para elegir el bandwidth con cross-validation (CV) que explicaremos en mayor detalle en la clase 8 tutorial 10.

In [ ]:
# Tarea para la casa: Mostrar el sesgo del histograma y esta funcion verdadera.

## Kernels con Seaborn
Continuaremos con el ejemplo utilizando la base de datos de propinas del modulo de `seaborn`. Para más información ver [seaborn](https://seaborn.pydata.org/)
La función de seaborn para graficar Kernels es [kdeplot()](https://seaborn.pydata.org/generated/seaborn.kdeplot.html).

In [ ]:
# Importamos la base de datos de propinas
tips = sns.load_dataset("tips")
tips

In [ ]:
# veamos la estadistica descriptiva por grupo
tips.groupby('sex').describe().round(2).T

In [ ]:
# Estimamos y graficamos la función por Kernels
sns.kdeplot(data=tips, x='tip') 

Nuevamente podemos mejorar el gráfico de seaborn usando las opciones de matplotlib

In [ ]:
sns.kdeplot(data=tips, x='tip')  # funcion de kernel de Seaborn
mean_tips = np.mean(tips['tip'])
plt.axvline(mean_tips, color='red', linestyle='dashed', linewidth=1, label='Tips promedio')
plt.title("Distribución de propinas (en USD)")
plt.xlabel("Propinas (en USD)")
plt.legend()  # Nos muestra la leyenda para la media de tips

También podemos hacer el grafico original del histograma (ver clase 8) y sumar con la opcion `kde=True`

In [ ]:
sns.histplot(data=tips['tip'], stat='density', kde=True) # funcion de histograma de Seaborn
mean_tips = np.mean(tips['tip'])
plt.axvline(mean_tips, color='red', linestyle='dashed', linewidth=1, label='Tips promedio')
plt.title("Distribución de propinas (en USD)")
plt.xlabel("Propinas (en USD)")
plt.legend()  # Nos muestra la leyenda para la media de tips

Nuevamente, podemos comparar las distribuciones de densidad de kernel entre hombres y mujeres


In [ ]:
# Checkeando el promedio de propinas
mean_tips_male = tips[tips['sex'] == 'Male']['tip'].mean()
print(mean_tips_male.round(2))
mean_tips_female = tips[tips['sex'] == 'Female']['tip'].mean()
print(mean_tips_female.round(2))

In [ ]:
sns.kdeplot(data=tips, x='tip',hue="sex",  multiple="stack", )  # funcion de kernel de Seaborn

plt.axvline(mean_tips_male, color='blue', linestyle='dashed', linewidth=1, label='Male')
plt.axvline(mean_tips_female, color='red', linestyle='dashed', linewidth=1, label='Female')

plt.title("Distribución de propinas (en USD)")
plt.xlabel("Propinas (en USD)")
plt.legend()  # Nos muestra la leyenda para la media de tips

In [ ]:
# tarea para la casa: jugar con la opcion de ver este grafico en dos paneles y sumarle el valor del promedio de cada grupo